In [ ]:
!pip install torchtext==0.6.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.2 MB/s eta 0:00:00
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.16.0
    Uninstalling torchtext-0.16.0:
      Successfully uninstalled torchtext-0.16.0


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchtext import data
from torchtext import datasets
from torchtext.data import Field, LabelField
import random

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
torch.manual_seed(42)
torch.backends.cudnn.deterministic = True

TEXT = Field(tokenize='spacy',
             tokenizer_language = 'en_core_web_sm',
             lower=True,
             include_lengths=True)
LABEL = LabelField(dtype=torch.float)

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:15<00:00, 5.41MB/s]


In [ ]:
train_data, valid_data = train_data.split(random_state = random.seed(42))

In [ ]:
vocab_size = 20000

TEXT.build_vocab(train_data,
                 max_size=vocab_size,
                 vectors="glove.6B.100d",
                 unk_init=torch.Tensor.normal_)

LABEL.build_vocab(train_data)

.vector_cache/glove.6B.zip: 862MB [02:42, 5.32MB/s]                           
100%|█████████▉| 399999/400000 [00:27<00:00, 14573.92it/s]


In [ ]:
batch_size = 64

train_loader, valid_loader, test_loader = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size=batch_size,
    sort_within_batch=True,
    device=device
)

In [ ]:
# class IMDbModel(nn.Module):
#   def __init__(self, vocab_size, embedding_dim, hidden_size, output_size):
#     super(IMDbModel, self).__init__()

#     self.embedding = nn.Embedding(vocab_size, embedding_dim)
#     self.lstm = nn.LSTM(embedding_dim, hidden_size, batch_first=True)
#     self.fc = nn.Linear(hidden_size, output_size)
#     self.sigmoid = nn.Sigmoid()

#   def forward(self, x, lengths):
#     embedded = self.embedding(x)

#     actual_lengths = torch.sum(x != 1, dim=1).cpu()

#     packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, actual_lengths, batch_first=True, enforce_sorted=False)

#     lstm_out, _ = self.lstm(packed_embedded)

#     unpacked_lstm_out, _ = nn.utils.rnn.pad_packed_sequence(lstm_out, batch_first=True)

#     last_output = unpacked_lstm_out[torch.arange(unpacked_lstm_out.size(0)), actual_lengths - 1]

#     output = self.fc(last_output)
#     output = self.sigmoid(output)
#     return output

In [ ]:
# class IMDbModel(nn.Module):
#   def __init__(self, vocab_size, embedding_dim, hidden_size, output_size):
#     super(IMDbModel, self).__init__()

#     self.embedding = nn.Embedding(vocab_size, embedding_dim)
#     self.dropout = nn.Dropout(0.5)
#     self.lstm = nn.LSTM(embedding_dim, hidden_size, batch_first=True, dropout=0.5)
#     self.conv1d = nn.Conv1d(hidden_size, 128, kernel_size=3, padding=1)
#     self.relu = nn.ReLU()
#     self.maxpool = nn.MaxPool1d(2)

#     conv_output_size = 128
#     maxpool_output_size = conv_output_size // 2

#     self.fc1 = nn.Linear(maxpool_output_size, 64)
#     self.fc2 = nn.Linear(64, output_size)
#     self.sigmoid = nn.Sigmoid()

#   def forward(self, x, lengths):
#     embedded = self.embedding(x)

#     actual_lengths = torch.sum(x != 1, dim=1).cpu()

#     packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, actual_lengths, batch_first=True, enforce_sorted=False)

#     lstm_out, _ = self.lstm(packed_embedded)

#     unpacked_lstm_out, _ = nn.utils.rnn.pad_packed_sequence(lstm_out, batch_first=True)

#     last_output = unpacked_lstm_out[torch.arange(unpacked_lstm_out.size(0)), actual_lengths - 1]

#     last_output = self.dropout(last_output)

#     last_output = last_output.unsqueeze(2)
#     last_output = self.conv1d(last_output)
#     last_output = self.relu(last_output)
#     last_output = self.maxpool(last_output.squeeze(2))

#     last_output = self.fc1(last_output)
#     last_output = self.relu(last_output)

#     output = self.fc2(last_output)
#     output = self.sigmoid(output)

#     return output

In [ ]:
import torch.nn as nn

class LSTM(nn.Module):
  def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers,
                bidirectional, dropout, pad_idx):

    super().__init__()

    self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)

    self.lstm = nn.LSTM(embedding_dim,
                        hidden_dim,
                        num_layers=n_layers,
                        bidirectional=bidirectional,
                        dropout=dropout)

    self.fc = nn.Linear(hidden_dim * 2, output_dim)

    self.dropout = nn.Dropout(dropout)

  def forward(self, text, text_lengths):
    embedded = self.dropout(self.embedding(text))

    # pack sequence
    # lengths need to be on CPU!
    packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.to('cpu'))

    packed_output, (hidden, cell) = self.lstm(packed_embedded)

    output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)

    # output = [sent len, batch size, hid dim * num directions]
    # output over padding tokens are zero tensors

    # hidden = [num layers * num directions, batch size, hid dim]
    # cell = [num layers * num directions, batch size, hid dim]

    # concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers
    # and apply dropout
    hidden = self.dropout(torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1))

    # hidden = [batch size, hid dim * num directions]

    # Apply sigmoid activation function
    output = torch.sigmoid(self.fc(hidden))

    return output

In [ ]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = LSTM(INPUT_DIM,
            EMBEDDING_DIM,
            HIDDEN_DIM,
            OUTPUT_DIM,
            N_LAYERS,
            BIDIRECTIONAL,
            DROPOUT,
            PAD_IDX)

In [ ]:
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
def count_parameters(model):
  return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 4,310,857 trainable parameters


In [ ]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([20002, 100])


In [ ]:
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[ 1.9269,  1.4873,  0.9007,  ...,  0.1233,  0.3499,  0.6173],
        [ 0.7262,  0.0912, -0.3891,  ...,  0.0821,  0.4440, -0.7240],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [ 0.1522,  0.3122, -0.8322,  ..., -0.2621, -0.7931,  0.4277],
        [ 0.3274, -1.5894, -1.4023,  ..., -0.6215,  2.3769,  0.5700],
        [ 1.0383, -0.4337, -2.1862,  ...,  1.4642,  1.1898,  0.8193]])

In [ ]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [ 0.1522,  0.3122, -0.8322,  ..., -0.2621, -0.7931,  0.4277],
        [ 0.3274, -1.5894, -1.4023,  ..., -0.6215,  2.3769,  0.5700],
        [ 1.0383, -0.4337, -2.1862,  ...,  1.4642,  1.1898,  0.8193]])


In [ ]:
def binary_accuracy(preds, y):
  """
  Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
  """

  rounded_preds = torch.round(torch.sigmoid(preds))
  correct = (rounded_preds == y).float()
  acc = correct.sum() / len(correct)
  return acc

In [ ]:
def train(model, iterator, optimizer, criterion):

  epoch_loss = 0
  epoch_acc = 0

  model.train()

  for batch in iterator:

    optimizer.zero_grad()

    text, text_lengths = batch.text

    predictions = model(text, text_lengths).squeeze(1)

    loss = criterion(predictions, batch.label)

    acc = binary_accuracy(predictions, batch.label)

    loss.backward()

    optimizer.step()

    epoch_loss += loss.item()
    epoch_acc += acc.item()

  return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):

  epoch_loss = 0
  epoch_acc = 0

  model.eval()

  with torch.no_grad():

    for batch in iterator:

      text, text_lengths = batch.text

      predictions = model(text, text_lengths).squeeze(1)

      loss = criterion(predictions, batch.label)

      acc = binary_accuracy(predictions, batch.label)

      epoch_loss += loss.item()
      epoch_acc += acc.item()

  return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 10

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss, train_acc = train(model, train_loader, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, test_loader, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')

    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

In [ ]:
model.load_state_dict(torch.load('tut2-model.pt'))

test_loss, test_acc = evaluate(model, test_loader, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')

def predict_sentiment(model, sentence):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.sigmoid(model(tensor, length_tensor))
    return prediction.item()

In [ ]:
predict_sentiment(model, "This film is terrible")

In [ ]:
predict_sentiment(model, "This film is great")